In [3]:
import faculty.datasets as datasets
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import spacy
import re
from utils import clean2, is_pua
from sentence_transformers import SentenceTransformer
import pickle
%matplotlib inline


2023-03-06 15:43:02.902120: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda/envs/Python3/lib:
2023-03-06 15:43:02.902186: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-06 15:43:02.902218: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cube-33199e64-1f46-4eab-ad41-37aeb8202adb-75d6f78f4d-6wp56): /proc/driver/nvidia/version does not exist


In [4]:
df = pd.read_excel('../data/raw/plans.xlsx')
df_missing = df[df['text'].isna()]
df_dirty = df[df['text'].notna()]

In [5]:
df_clean = df_dirty
df_clean['text'] = df_dirty['text'].map(clean2)
test_corpus = df_clean['text']

/tmp/ipykernel_122633/1577469100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['text'] = df_dirty['text'].map(clean2)


In [9]:
test_corpus

0      aberdeen city council energy and climate plan ...
1      aberdeen adapts aberdeens climate adaptation f...
2      council climate change plan 2021 2025 towardsn...
3      a climatepositive city at the heart of the glo...
5      carbon neutral plan working towards the 2030 t...
                             ...                        
668    wychavon 20202030 intelligently green plan con...
670    climate change action plan. council operations...
671    climate change action plan. wider borough our ...
680    restore revive thrive our environment climate ...
685    1 zero carbon london.1.5c compatible plan dece...
Name: text, Length: 502, dtype: object

AttributeError: module 'spacy' has no attribute 'download'

In [1]:
# lemmatize(test_corpus[5])

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

my_stop_words = text.ENGLISH_STOP_WORDS
#setting max_features = n gives the n most frequent bigrams across the corpus
tfidf_vectorizer = TfidfVectorizer(use_idf=True,tokenizer=lemmatize, stop_words="english", max_features=100,ngram_range=(2,2))#hyperparameter to explore


tfidf_vectors = tfidf_vectorizer.fit_transform(df_clean['text'])

/opt/anaconda/envs/Python3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'make', 'whereaft'] not in stop_words.
  warnings.warn(


In [24]:
# print out your vectors in a nice way
features = tfidf_vectorizer.get_feature_names()
for i, row in enumerate(tfidf_vectors.toarray()):
    # print out your vectors in a nice way
    print(
        pd.DataFrame({"Terms": features, "Frq_id_freq": row}).to_string(index=False)
    )
    print("-" * 40)

/opt/anaconda/envs/Python3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


                  Terms  Frq_id_freq
                            0.000000
                     xc     0.000000
                 action     0.068914
                 carbon     0.000000
                climate     0.114886
                council     0.000000
               emission     0.000000
                 energy     0.000000
               increase     0.075947
                 reduce     0.000000
              reduction     0.000000
                  scope     0.000000
                support     0.076163
                 target     0.000000
                  tonne     0.000000
                   work     0.000000
                     xc     0.369528
                   year     0.000000
            achieve net     0.000000
               action       0.000000
            action plan     0.043102
          active travel     0.159623
            air quality     0.000000
             appendix       0.000000
        borough council     0.000000
             building       0.000000
 

In [25]:
features = tfidf_vectorizer.get_feature_names()
features

/opt/anaconda/envs/Python3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['   ',
 '   xc',
 '  action',
 '  carbon',
 '  climate',
 '  council',
 '  emission',
 '  energy',
 '  increase',
 '  reduce',
 '  reduction',
 '  scope',
 '  support',
 '  target',
 '  tonne',
 '  work',
 '  xc',
 '  year',
 'achieve net',
 'action  ',
 'action plan',
 'active travel',
 'air quality',
 'appendix  ',
 'borough council',
 'building  ',
 'carbon budget',
 'carbon dioxide',
 'carbon emission',
 'carbon footprint',
 'carbon management',
 'carbon neutral',
 'carbon neutrality',
 'carbon reduction',
 'change  ',
 'change action',
 'change climate',
 'change strategy',
 'city council',
 'climate action',
 'climate change',
 'climate emergency',
 'council  ',
 'council building',
 'council carbon',
 'council climate',
 'county council',
 'declare climate',
 'district council',
 'electric vehicle',
 'emergency action',
 'emission  ',
 'emission council',
 'emission reduction',
 'energy  ',
 'energy efficiency',
 'energy efficient',
 'energy use',
 'figure  ',
 'fossil fuel',
 

In [14]:
len(features)

100

In [271]:
# pip install -U sentence-transformers

In [15]:
import os
os.environ["HF_ENDPOINT"] = "https://huggingface.co"

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("whaleloops/phrase-bert")

In [16]:
from scipy.spatial.distance import cdist

In [17]:
phrase_embs = model.encode(features) #returns np array shape (200,768)

dot_similarity = np.dot(phrase_embs,phrase_embs.T)
# np.fill_diagonal(dot_similarity,0)
cos_similarity = 1- cdist(phrase_embs,phrase_embs, 'cosine')
# np.fill_diagonal(cos_similarity,0)

In [18]:
dot_df =pd.DataFrame(
    dot_similarity,
    index=features,
    columns=features,
)

cos_df =pd.DataFrame(
    cos_similarity,
    index=features,
    columns=features,
)

In [21]:

# data =cos_df.loc['achieve carbon'].sort_values(ascending=False).index.to_list()

h= pd.DataFrame(index=range(100),columns = cos_df.index.to_list())

for col in h.columns:
    h[col] = cos_df.loc[col].sort_values(ascending=False).index.to_list()

In [22]:

sim_words = h.transpose()
sim_words.to_csv('../data/processed/model1_100similar_phrases.csv')




## Thoughts on baseline
- Produces too many overlapping phrases. e.g. council declare and council declare climate, strategy action and strategy action plan
- Produces opposite phrases. e.g long term and short term, private sector and public sector. People won't want to search for both
- Produces nonsense like protect enhance, action reduce